In [1]:
from pymatsolver import Pardiso
import numpy as np
import scipy as sp
import dask
import dask.array as da

In [4]:
nC = 100000

# Solve it for a random RHS
b = np.random.randn(nC,100)

# Create a simple sparse banded system

A = sp.sparse.diags([-np.ones(nC), 2*np.ones(nC), -np.ones(nC)], [-1,0,1], shape=(nC,nC))

# Create Pardiso solver for Ainv
Ainv = Pardiso(A)

In [5]:
# Solve the RHS in a loop using Solver
y = Ainv*b

In [7]:
y

array([[  -83.04469337,   125.29141463,   141.31561004, ...,
           50.86037676,    57.20597834,  -181.63176232],
       [ -165.58224538,   249.78937907,   282.52041176, ...,
          101.24368887,   114.77295993,  -363.46322324],
       [ -249.02643549,   376.6886822 ,   423.68469328, ...,
          150.97582329,   172.11243426,  -546.54214202],
       ...,
       [ 1059.38066037,   420.15659659,   409.37113333, ...,
           99.02414581,   512.33856505, -1218.57767749],
       [  706.00555312,   280.84104283,   272.35951447, ...,
           65.32718546,   341.56716421,  -810.4666193 ],
       [  353.35739267,   141.33042513,   135.85085976, ...,
           32.47032283,   171.78616999,  -404.76936579]])

In [6]:
# Try to do the same solves inside a delayed function
from pyMKL import mkl_set_num_threads

@dask.delayed
def solveRHD(rhs):
    
    # Create a simple sparse banded system
    nC = 100000
    A = sp.sparse.diags([-np.ones(nC), 2*np.ones(nC), -np.ones(nC)], [-1,0,1], shape=(nC,nC))

    # Create Pardiso solver for Ainv
    Ainv = Pardiso(A)    
    mkl_set_num_threads(1)
    return Ainv*rhs

# Split the solves in dask delays
columns = [solveRHD(b[:, ii]).reshape((nC,1)) for ii in range(b.shape[1])]
solves = [da.from_delayed(column, dtype='float', shape=(nC, 1)).reshape((nC,1)) for column in columns]

# Stack the result
yDask = da.hstack(solves)

In [15]:
# Compute only one of them 
# First is the same ... good
# solves[0].compute()

In [8]:
# Run all with dask ... not good, run again different
y_dask = yDask.compute()

In [9]:
(y_dask - y) / y


array([[ 2.63137505e-11,  1.91653029e-10,  1.18829847e-10, ...,
         2.79247094e-11,  2.09756131e-10,  1.10961108e-10],
       [ 2.63943436e-11,  1.92261810e-10,  1.18876453e-10, ...,
         2.80562919e-11,  2.09096370e-10,  1.10900142e-10],
       [ 2.63251820e-11,  1.91238656e-10,  1.18903368e-10, ...,
         2.82216294e-11,  2.09153477e-10,  1.10626757e-10],
       ...,
       [-1.87371005e-13,  6.72382892e-11,  4.91033190e-11, ...,
         2.48514510e-12,  2.34450635e-11,  1.72787380e-11],
       [-1.87436881e-13,  6.70619908e-11,  4.92031634e-11, ...,
         2.51142486e-12,  2.34445061e-11,  1.73196862e-11],
       [-1.87248777e-13,  6.66302370e-11,  4.93222851e-11, ...,
         2.52637337e-12,  2.33076780e-11,  1.73395379e-11]])